In [25]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp', node_spec=NODE_LARGE)

Spark session already exists. Return it
If there are no Spark clusters, please do sphynx.stop() first


In [3]:
# user 지역
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

In [4]:
def pre_get_user_and_gcoin(event_name, device, start_date, end_date):
    user = pre_load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='user_master').where(col("lastlogindate") >= start_date)
    user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
    user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")
    gcoin = pre_load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='gcoin_use').where(col("event_name") == event_name)
    return user, gcoin

In [5]:
def get_user_and_gcoin(event_name, device, start_date, end_date):
    user = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='user_master').where(col("lastlogindate") >= start_date)
    user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
    user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")
    gcoin = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='gcoin_use').where(col("event_name") == event_name)
    return user, gcoin

### EMOTE MAR2

In [21]:
pc_start_date = "2021-03-13"
pc_end_date = "2021-06-12"

console_start_date = "2021-03-18"
console_end_date = "2021-06-17"

event_name = "2021_emote_mar2"

PC

In [22]:
mar2_user_pc, mar2_gcoin_pc = pre_get_user_and_gcoin(event_name = event_name, device = "pc", start_date = pc_start_date, end_date = pc_end_date)

In [24]:
mar2_user_pc_tmp = mar2_user_pc.select("accountid", "pubg_region").distinct()

In [30]:
mar2_gcoin_pc_tmp = mar2_gcoin_pc.join(mar2_user_pc_tmp, mar2_gcoin_pc.account_id == mar2_user_pc_tmp.accountid,"left")

In [31]:
mar2_pc_a = mar2_user_pc.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
mar2_pc_b = mar2_gcoin_pc_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [32]:
mar2_pc = pd.merge(mar2_pc_a, mar2_pc_b, on='pubg_region', how="left").fillna(0)
mar2_pc['pur'] = mar2_pc.pu / mar2_pc.au
mar2_pc['arpu'] = mar2_pc.revenue / mar2_pc.au
mar2_pc['arppu'] = mar2_pc.revenue / mar2_pc.pu

In [33]:
mar2_pc

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,7186110,84936.0,84936.0,50961600.0,0.011819,7.091681,600.0
1,NA,359627,1801.0,1801.0,1080600.0,0.005008,3.004780,600.0
2,0,2,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,194770,353.0,353.0,211800.0,0.001812,1.087436,600.0
4,KR,2203213,28066.0,28066.0,16839600.0,0.012739,7.643201,600.0
5,SEA,497195,1388.0,1388.0,832800.0,0.002792,1.674997,600.0
6,Undefined,1541,2.0,2.0,1200.0,0.001298,0.778715,600.0
7,JP,63244,317.0,317.0,190200.0,0.005012,3.007400,600.0
8,TW/HK,129184,978.0,978.0,586800.0,0.007571,4.542358,600.0
9,CIS,466349,858.0,858.0,514800.0,0.001840,1.103894,600.0


CONSOLE

In [34]:
mar2_user_console, mar2_gcoin_console = pre_get_user_and_gcoin(event_name=event_name, device="console", start_date=console_start_date, end_date=console_end_date)

In [35]:
mar2_user_console_tmp = mar2_user_console.select("accountid", "pubg_region").distinct()
mar2_gcoin_console_tmp = mar2_gcoin_console.join(mar2_user_console_tmp, mar2_gcoin_console.account_id == mar2_user_console_tmp.accountid,"left")

In [37]:
mar2_console_a = mar2_user_console.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
print("---")
mar2_console_b = mar2_gcoin_console_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

---


In [38]:
mar2_console = pd.merge(mar2_console_a, mar2_console_b, on='pubg_region', how="left").fillna(0)
mar2_console['pur'] = mar2_console.pu / mar2_console.au
mar2_console['arpu'] = mar2_console.revenue / mar2_console.au
mar2_console['arppu'] = mar2_console.revenue / mar2_console.pu

In [39]:
mar2_console

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,13353,18.0,18.0,10800.0,0.001348,0.808807,600.0
1,NA,1111353,7673.0,7673.0,4603800.0,0.006904,4.142518,600.0
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,309000,486.0,486.0,291600.0,0.001573,0.943689,600.0
4,KR,11349,169.0,169.0,101400.0,0.014891,8.934708,600.0
5,SEA,31428,68.0,68.0,40800.0,0.002164,1.298205,600.0
6,Undefined,1976,0.0,0.0,0.0,0.000000,0.000000,NaN
7,JP,38650,555.0,555.0,333000.0,0.014360,8.615783,600.0
8,TW/HK,10142,80.0,80.0,48000.0,0.007888,4.732794,600.0
9,CIS,58188,79.0,79.0,47400.0,0.001358,0.814601,600.0


## Elsolitario

In [40]:
pc_start_date = "2021-06-02"
pc_end_date = "2021-10-22"

console_start_date = "2021-06-10"
console_end_date = "2021-10-22"

event_name = '202106_elsolitario'

PC

In [41]:
elso_user_pc, elso_gcoin_pc = pre_get_user_and_gcoin(event_name = event_name, device = "pc", start_date = pc_start_date, end_date = pc_end_date)

In [42]:
elso_user_pc_tmp = elso_user_pc.select("accountid", "pubg_region").distinct()

In [43]:
elso_gcoin_pc_tmp = elso_gcoin_pc.join(elso_user_pc_tmp, elso_gcoin_pc.account_id == elso_user_pc_tmp.accountid,"left")

In [44]:
elso_pc_a = elso_user_pc.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
elso_pc_b = elso_gcoin_pc_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [45]:
elso_pc = pd.merge(elso_pc_a, elso_pc_b, on='pubg_region', how="left").fillna(0)
elso_pc['pur'] = elso_pc.pu / elso_pc.au
elso_pc['arpu'] = elso_pc.revenue / elso_pc.au
elso_pc['arppu'] = elso_pc.revenue / elso_pc.pu

In [46]:
elso_pc

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,9371121,114659.0,94307.0,69871710.0,0.010064,7.456067,740.896328
1,NA,462525,3589.0,2932.0,2222070.0,0.006339,4.804216,757.868349
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,325089,1255.0,1110.0,583430.0,0.003414,1.794678,525.612613
4,KR,2821251,21627.0,18140.0,19091180.0,0.006430,6.766920,1052.435502
5,SEA,659151,4807.0,4208.0,2358330.0,0.006384,3.577830,560.439639
6,Undefined,10892,10.0,9.0,4090.0,0.000826,0.375505,454.444444
7,JP,85263,438.0,357.0,288920.0,0.004187,3.388574,809.299720
8,TW/HK,172395,1002.0,838.0,638760.0,0.004861,3.705212,762.243437
9,CIS,784131,2628.0,2323.0,1288800.0,0.002963,1.643603,554.799828


CONSOLE

In [47]:
elso_user_console, elso_gcoin_console = pre_get_user_and_gcoin(event_name=event_name, device="console", start_date=console_start_date, end_date=console_end_date)

In [48]:
elso_user_console_tmp = elso_user_console.select("accountid", "pubg_region").distinct()
elso_gcoin_console_tmp = elso_gcoin_console.join(elso_user_console_tmp, elso_gcoin_console.account_id == elso_user_console_tmp.accountid,"left")

In [49]:
elso_console_a = elso_user_console.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
print("---")
elso_console_b = elso_gcoin_console_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

---


In [50]:
elso_console = pd.merge(elso_console_a, elso_console_b, on='pubg_region', how="left").fillna(0)
elso_console['pur'] = elso_console.pu / elso_console.au
elso_console['arpu'] = elso_console.revenue / elso_console.au
elso_console['arppu'] = elso_console.revenue / elso_console.pu

In [51]:
elso_console

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,22539,27.0,23.0,16870.0,0.001020,0.748480,733.478261
1,NA,1373359,10900.0,8705.0,7081190.0,0.006338,5.156110,813.462378
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,378443,648.0,561.0,390410.0,0.001482,1.031622,695.918004
4,KR,15566,77.0,63.0,64790.0,0.004047,4.162277,1028.412698
5,SEA,32647,64.0,47.0,53160.0,0.001440,1.628327,1131.063830
6,Undefined,2642,4.0,1.0,2090.0,0.000379,0.791067,2090.000000
7,JP,45774,337.0,290.0,243350.0,0.006335,5.316337,839.137931
8,TW/HK,13240,43.0,38.0,37680.0,0.002870,2.845921,991.578947
9,CIS,66700,126.0,110.0,87960.0,0.001649,1.318741,799.636364


## Phao

In [30]:
pc_start_date = "2021-07-07"
pc_end_date = "2021-10-06"

console_start_date = "2021-07-15"
console_end_date = "2021-10-14"

event_name = "202107_phao_emote"

PC

In [53]:
phao_user_pc, phao_gcoin_pc = pre_get_user_and_gcoin(event_name = event_name, device = "pc", start_date = pc_start_date, end_date = pc_end_date)

In [54]:
phao_user_pc_tmp = phao_user_pc.select("accountid", "pubg_region").distinct()

In [55]:
phao_gcoin_pc_tmp = phao_gcoin_pc.join(phao_user_pc_tmp, phao_gcoin_pc.account_id == phao_user_pc_tmp.accountid,"left")

In [56]:
phao_pc_a = phao_user_pc.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
phao_pc_b = phao_gcoin_pc_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [57]:
phao_pc = pd.merge(phao_pc_a, phao_pc_b, on='pubg_region', how="left").fillna(0)
phao_pc['pur'] = phao_pc.pu / phao_pc.au
phao_pc['arpu'] = phao_pc.revenue / phao_pc.au
phao_pc['arppu'] = phao_pc.revenue / phao_pc.pu

In [58]:
phao_pc

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,7898848,229714.0,229714.0,137828400.0,0.029082,17.449177,600.000000
1,NA,364485,5654.0,5654.0,3392400.0,0.015512,9.307379,600.000000
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,272446,1275.0,1275.0,765000.0,0.004680,2.807896,600.000000
4,KR,2294155,109466.0,109465.0,65679600.0,0.047715,28.629103,600.005481
5,SEA,546266,6635.0,6635.0,3981000.0,0.012146,7.287658,600.000000
6,Undefined,9504,18.0,18.0,10800.0,0.001894,1.136364,600.000000
7,JP,66293,792.0,792.0,475200.0,0.011947,7.168178,600.000000
8,TW/HK,133625,3025.0,3025.0,1815000.0,0.022638,13.582788,600.000000
9,CIS,658354,4837.0,4837.0,2902200.0,0.007347,4.408267,600.000000


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56478)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

CONSOLE

In [31]:
phao_user_console, phao_gcoin_console = get_user_and_gcoin(event_name=event_name, device="console", start_date=console_start_date, end_date=console_end_date)

In [32]:
phao_user_console_tmp = phao_user_console.select("accountid", "pubg_region").distinct()
phao_gcoin_console_tmp = phao_gcoin_console.join(phao_user_console_tmp, phao_gcoin_console.account_id == phao_user_console_tmp.accountid,"left")

In [33]:
phao_console_a = phao_user_console.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
print("---")
phao_console_b = phao_gcoin_console_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

---


In [34]:
phao_console = pd.merge(phao_console_a, phao_console_b, on='pubg_region', how="left").fillna(0)
phao_console['pur'] = phao_console.pu / phao_console.au
phao_console['arpu'] = phao_console.revenue / phao_console.au
phao_console['arppu'] = phao_console.revenue / phao_console.pu

In [35]:
phao_console

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,16714,100.0,100.0,60000.0,0.005983,3.589805,600.0
1,NA,1077722,10778.0,10778.0,6466800.0,0.010001,6.000434,600.0
2,SA,286784,899.0,899.0,539400.0,0.003135,1.880858,600.0
3,KR,12598,272.0,272.0,163200.0,0.021591,12.954437,600.0
4,SEA,23484,136.0,136.0,81600.0,0.005791,3.474706,600.0
5,Undefined,1924,0.0,0.0,0.0,0.000000,0.000000,NaN
6,JP,35724,610.0,610.0,366000.0,0.017075,10.245213,600.0
7,TW/HK,10034,128.0,128.0,76800.0,0.012757,7.653976,600.0
8,CIS,50530,196.0,196.0,117600.0,0.003879,2.327330,600.0
9,EMEA,510360,3165.0,3165.0,1899000.0,0.006202,3.720903,600.0


## SON

In [6]:
pc_start_date = "2021-07-21"
pc_end_date = "2021-10-20"

console_start_date = "2021-07-21"
console_end_date = "2021-10-20"

event_name = "202107_son"

PC

In [7]:
son_user_pc, son_gcoin_pc = pre_get_user_and_gcoin(event_name = event_name, device = "pc", start_date = pc_start_date, end_date = pc_end_date)
son_user_pc_tmp = son_user_pc.select("accountid", "pubg_region").distinct()
son_gcoin_pc_tmp = son_gcoin_pc.join(son_user_pc_tmp, son_gcoin_pc.account_id == son_user_pc_tmp.accountid,"left")

In [8]:
son_pc_a = son_user_pc.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
son_pc_b = son_gcoin_pc_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [9]:
son_pc = pd.merge(son_pc_a, son_pc_b, on='pubg_region', how="left").fillna(0)
son_pc['pur'] = son_pc.pu / son_pc.au
son_pc['arpu'] = son_pc.revenue / son_pc.au
son_pc['arppu'] = son_pc.revenue / son_pc.pu

In [10]:
son_pc

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,7600681,116727.0,100906.0,119506700.0,0.013276,15.723157,1184.336908
1,NA,340774,5249.0,3933.0,5111580.0,0.011541,14.999912,1299.664378
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,260946,2188.0,1812.0,2079360.0,0.006944,7.968545,1147.549669
4,KR,2214992,38101.0,31438.0,59932210.0,0.014193,27.057529,1906.362046
5,SEA,524818,10737.0,9189.0,12412290.0,0.017509,23.650656,1350.777016
6,Undefined,9470,23.0,20.0,22060.0,0.002112,2.329461,1103.000000
7,JP,60446,617.0,517.0,651700.0,0.008553,10.781524,1260.541586
8,TW/HK,125838,1563.0,1323.0,1773680.0,0.010514,14.094947,1340.650038
9,CIS,642211,3548.0,2875.0,2933460.0,0.004477,4.567751,1020.333913


CONSOLE

In [11]:
son_user_console, son_gcoin_console = pre_get_user_and_gcoin(event_name=event_name, device="console", start_date=console_start_date, end_date=console_end_date)
son_user_console_tmp = son_user_console.select("accountid", "pubg_region").distinct()
son_gcoin_console_tmp = son_gcoin_console.join(son_user_console_tmp, son_gcoin_console.account_id == son_user_console_tmp.accountid,"left")

In [12]:
son_console_a = son_user_console.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
print("---")
son_console_b = son_gcoin_console_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

---


In [13]:
son_console = pd.merge(son_console_a, son_console_b, on='pubg_region', how="left").fillna(0)
son_console['pur'] = son_console.pu / son_console.au
son_console['arpu'] = son_console.revenue / son_console.au
son_console['arppu'] = son_console.revenue / son_console.pu

In [20]:
son_console

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,16075,41,38,45540,0.002364,2.832970,1198.421053
1,NA,1051971,16958,11072,12975850,0.010525,12.334798,1171.951770
2,SA,283637,1520,1163,1333070,0.004100,4.699916,1146.233878
3,KR,12328,122,92,156900,0.007463,12.727125,1705.434783
4,SEA,22920,105,80,114420,0.003490,4.992147,1430.250000
5,Undefined,1943,1,1,990,0.000515,0.509521,990.000000
6,JP,34922,377,298,364900,0.008533,10.449001,1224.496644
7,TW/HK,9745,45,37,55620,0.003797,5.707542,1503.243243
8,CIS,50263,213,140,164680,0.002785,3.276366,1176.285714
9,EMEA,501279,3772,2693,3292790,0.005372,6.568777,1222.721872


## BLACKPINK

In [15]:
pc_start_date = "2021-08-04"
pc_end_date = "2021-10-22"

console_start_date = "2021-08-12"
console_end_date = "2021-10-22"

event_name = "202108_blackpink"

PC

In [17]:
bp_user_pc, bp_gcoin_pc = get_user_and_gcoin(event_name = event_name, device = "pc", start_date = pc_start_date, end_date = pc_end_date)
bp_user_pc_tmp = bp_user_pc.select("accountid", "pubg_region").distinct()
bp_gcoin_pc_tmp = bp_gcoin_pc.join(bp_user_pc_tmp, bp_gcoin_pc.account_id == bp_user_pc_tmp.accountid,"left")

In [18]:
bp_pc_a = bp_user_pc.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
bp_pc_b = bp_gcoin_pc_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

In [21]:
bp_pc = pd.merge(bp_pc_a, bp_pc_b, on='pubg_region', how="left").fillna(0)
bp_pc['pur'] = bp_pc.pu / bp_pc.au
bp_pc['arpu'] = bp_pc.revenue / bp_pc.au
bp_pc['arppu'] = bp_pc.revenue / bp_pc.pu

In [22]:
bp_pc

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,7047820,174701.0,154005.0,890106140.0,0.021851,126.295243,5779.722347
1,NA,305250,5046.0,4385.0,23312360.0,0.014365,76.371368,5316.387685
2,0,1,0.0,0.0,0.0,0.000000,0.000000,NaN
3,SA,243959,590.0,522.0,2323720.0,0.002140,9.525043,4451.570881
4,KR,2040248,84409.0,74870.0,375284140.0,0.036697,183.940452,5012.476826
5,SEA,487182,9682.0,8737.0,43462840.0,0.017934,89.212738,4974.572508
6,Undefined,8827,21.0,21.0,93980.0,0.002379,10.646879,4475.238095
7,JP,53126,1048.0,905.0,5227440.0,0.017035,98.397018,5776.176796
8,TW/HK,111896,3203.0,2822.0,17320180.0,0.025220,154.788196,6137.554926
9,CIS,603245,1498.0,1271.0,5118520.0,0.002107,8.484977,4027.159717


CONSOLE

In [23]:
bp_user_console, bp_gcoin_console = get_user_and_gcoin(event_name=event_name, device="console", start_date=console_start_date, end_date=console_end_date)
bp_user_console_tmp = bp_user_console.select("accountid", "pubg_region").distinct()
bp_gcoin_console_tmp = bp_gcoin_console.join(bp_user_console_tmp, bp_gcoin_console.account_id == bp_user_console_tmp.accountid,"left")

In [27]:
bp_console_a = bp_user_console.groupBy('pubg_region').agg(countDistinct('accountid').alias('au')).toPandas()
print("---")
bp_console_b = bp_gcoin_console_tmp.groupBy("pubg_region").agg(count("*").alias("cnt"), countDistinct("account_id").alias("pu"), sum("price").alias("revenue")).toPandas()

---


In [28]:
bp_console = pd.merge(bp_console_a, bp_console_b, on='pubg_region', how="left").fillna(0)
bp_console['pur'] = bp_console.pu / bp_console.au
bp_console['arpu'] = bp_console.revenue / bp_console.au
bp_console['arppu'] = bp_console.revenue / bp_console.pu

In [29]:
bp_console

,pubg_region,au,cnt,pu,revenue,pur,arpu,arppu
0,CN,11834,74.0,62.0,326760.0,0.005239,27.611966,5270.322581
1,NA,836535,6402.0,5203.0,25672280.0,0.006220,30.688830,4934.130309
2,SA,226637,345.0,289.0,1300860.0,0.001275,5.739839,4501.245675
3,KR,9690,172.0,149.0,762960.0,0.015377,78.736842,5120.536913
4,SEA,17389,93.0,79.0,429420.0,0.004543,24.694922,5435.696203
5,Undefined,1497,0.0,0.0,0.0,0.000000,0.000000,NaN
6,JP,28417,776.0,646.0,3794880.0,0.022733,133.542598,5874.427245
7,TW/HK,7569,119.0,105.0,558660.0,0.013872,73.808958,5320.571429
8,CIS,40850,42.0,37.0,123880.0,0.000906,3.032558,3348.108108
9,EMEA,393176,1736.0,1447.0,6655920.0,0.003680,16.928602,4599.806496
